In [7]:
import pandas as pd 
import numpy as np
import requests #used for HHTP requests and API calls
import re
import sqlite3

In [8]:
# Save cleaned dataframes

df_train = pd.read_csv(r"Resources/cleaned_data.csv")

# Cleaned df_train

In [9]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)
# Reset the index
df_train.reset_index(drop=True, inplace=True)
# Display the DataFrame
df_train

,customer_id,month,name,age,occupation,annual_income,num_bank_accounts,num_of_loans,type_of_loans,delay_from_due_date,num_of_delayed_payments,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,CUS_0xd40,April,Aaron Maashoh,24,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4.0,6,4.0,Good,809.98,31.38,22 Years and 4 Months,No,49.57,199.46,Low_spent_Small_value_payments,223.45,Good
1,CUS_0xd40,July,Aaron Maashoh,24,Scientist,19114.12,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8.0,11,4.0,Good,809.98,22.54,22 Years and 7 Months,No,49.57,178.34,Low_spent_Small_value_payments,244.57,Good
2,CUS_0x21b1,January,Rick Rothackerj,28,NaN,34847.84,2,1,Credit-Builder Loan,3,4.0,5,2.0,Good,605.03,24.46,26 Years and 7 Months,No,18.82,104.29,Low_spent_Small_value_payments,470.69,Standard
3,CUS_0x21b1,February,Rick Rothackerj,28,Teacher,34847.84,2,1,Credit-Builder Loan,7,1.0,7,2.0,Good,605.03,38.55,26 Years and 8 Months,No,18.82,40.39,High_spent_Large_value_payments,484.59,Good
4,CUS_0x21b1,April,Rick Rothackerj,28,Teacher,34847.84,2,1,Credit-Builder Loan,3,3.0,5,2.0,Good,605.03,39.18,26 Years and 10 Months,No,18.82,99.31,Low_spent_Medium_value_payments,465.68,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50107,CUS_0x8600,January,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",35,26.0,18,9.0,Bad,3571.70,22.90,5 Years and 8 Months,Yes,60.96,43.37,High_spent_Large_value_payments,328.66,Poor
50108,CUS_0x8600,February,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,23,9.0,Bad,3571.70,39.77,5 Years and 9 Months,Yes,12112.00,148.28,Low_spent_Small_value_payments,273.75,Poor
50109,CUS_0x8600,March,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,18,9.0,Bad,3571.70,24.71,5 Years and 10 Months,Yes,60.96,76.28,Low_spent_Small_value_payments,345.74,Poor
50110,CUS_0x8600,August,Sarah McBridec,4975,Architect,20002.88,10,5,"Personal Loan, Auto Loan, Mortgage Loan, Stude...",33,25.0,18,9.0,Bad,3571.70,37.14,6 Years and 3 Months,Yes,60.96,34.66,High_spent_Large_value_payments,337.36,Standard


In [10]:
# Save cleaned dataframes

df_train.to_csv("Resources\cleaned_data.csv", index = False)

<>:3: SyntaxWarning: invalid escape sequence '\c'
<>:3: SyntaxWarning: invalid escape sequence '\c'
/var/folders/_1/6_ldv5g53dl0ywyb21ky4b7m0000gn/T/ipykernel_68314/3723894504.py:3: SyntaxWarning: invalid escape sequence '\c'
  df_train.to_csv("Resources\cleaned_data.csv", index = False)


In [11]:
# Connect to SQLite database
conn = sqlite3.connect('credit_data.db')
cursor = conn.cursor()

# Drop tables if they already exist to avoid duplicates
cursor.execute('DROP TABLE IF EXISTS personal_info')
cursor.execute('DROP TABLE IF EXISTS financial_info')


In [12]:
# Create personal_info table
cursor.execute('''
CREATE TABLE personal_info (
    customer_id TEXT PRIMARY KEY,
    name TEXT,
    age INTEGER,
    occupation TEXT,
    annual_income REAL
)
''')

# Create financial_info table
cursor.execute('''
CREATE TABLE financial_info (
    customer_id TEXT,
    num_bank_accounts INTEGER,
    num_of_loans INTEGER,
    credit_score TEXT,
    credit_utilization_ratio REAL,
    FOREIGN KEY (customer_id) REFERENCES personal_info (customer_id)
)
''')

In [13]:
# Insert data into the personal_info table
personal_data = df_train[['customer_id', 'name', 'age', 'occupation', 'annual_income']].drop_duplicates(subset=['customer_id'])
personal_data.to_sql('personal_info', conn, if_exists='append', index=False)

11062

In [14]:
# Insert data into the financial_info table
financial_data = df_train[['customer_id', 'num_bank_accounts', 'num_of_loans', 'credit_score', 'credit_utilization_ratio']]
financial_data.to_sql('financial_info', conn, if_exists='append', index=False)


50112

In [15]:
# Commit changes and close connection
conn.commit()


In [16]:
# Query to group by age group and count the credit scores
age_group_query = '''
SELECT 
    CASE 
        WHEN age BETWEEN 18 AND 25 THEN '18-25'
        WHEN age BETWEEN 26 AND 35 THEN '26-35'
        WHEN age BETWEEN 36 AND 45 THEN '36-45'
        WHEN age BETWEEN 46 AND 55 THEN '46-55'
        WHEN age BETWEEN 56 AND 65 THEN '56-65'
        ELSE '66-75' 
    END AS age_group,
    credit_score,
    COUNT(credit_score) AS count
FROM personal_info
JOIN financial_info ON personal_info.customer_id = financial_info.customer_id
GROUP BY age_group, credit_score
ORDER BY age_group
'''

# Execute the query and fetch results
cursor.execute(age_group_query)
result = cursor.fetchall()

# Display the results
for row in result:
    print(f"Age Group: {row[0]}, Credit Score: {row[1]}, Count: {row[2]}")


Age Group: 18-25, Credit Score: Good, Count: 1556
Age Group: 18-25, Credit Score: Poor, Count: 3564
Age Group: 18-25, Credit Score: Standard, Count: 5583
Age Group: 26-35, Credit Score: Good, Count: 2081
Age Group: 26-35, Credit Score: Poor, Count: 4716
Age Group: 26-35, Credit Score: Standard, Count: 7380
Age Group: 36-45, Credit Score: Good, Count: 2051
Age Group: 36-45, Credit Score: Poor, Count: 4576
Age Group: 36-45, Credit Score: Standard, Count: 7464
Age Group: 46-55, Credit Score: Good, Count: 1932
Age Group: 46-55, Credit Score: Poor, Count: 1044
Age Group: 46-55, Credit Score: Standard, Count: 3565
Age Group: 56-65, Credit Score: Good, Count: 175
Age Group: 56-65, Credit Score: Poor, Count: 50
Age Group: 56-65, Credit Score: Standard, Count: 331
Age Group: 66-75, Credit Score: Good, Count: 286
Age Group: 66-75, Credit Score: Poor, Count: 1670
Age Group: 66-75, Credit Score: Standard, Count: 2088


In [17]:
# Query to calculate average bank accounts and loans by credit score
effect_query = '''
SELECT 
    credit_score,
    AVG(num_bank_accounts) AS avg_bank_accounts,
    AVG(num_of_loans) AS avg_loans
FROM financial_info
GROUP BY credit_score
'''

# Execute the query and fetch results
cursor.execute(effect_query)
result = cursor.fetchall()

# Display the results
for row in result:
    print(f"Credit Score: {row[0]}, Avg Bank Accounts: {row[1]:.2f}, Avg Loans: {row[2]:.2f}")


Credit Score: Good, Avg Bank Accounts: 15.29, Avg Loans: 2.31
Credit Score: Poor, Avg Bank Accounts: 17.56, Avg Loans: 3.69
Credit Score: Standard, Avg Bank Accounts: 17.27, Avg Loans: 2.59


In [18]:
# Close the SQLite connection
conn.close()
